# NOW THE SAME PIPELINE USING PINECONE (Cloud, Production Grade)

## Install

In [ ]:
pip install pinecone-client sentence-transformers langchain

## Step 1: Initialize Pinecone

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="YOUR_API_KEY")
index = pc.Index("my-index")


## Step 2–3: Embed + Insert into Pinecone

In [ ]:
vectors = [
    (str(i), embeddings[i].tolist(), {"text": chunks[i]})
    for i in range(len(chunks))
]

index.upsert(vectors)


## Step 4: Query Embedding

In [ ]:
query_emb = model.encode([query]).tolist()[0]


## Step 5: Pinecone Similarity Search

In [ ]:
results = index.query(
    vector=query_emb,
    top_k=3,
    include_metadata=True
)

retrieved_chunks = [match["metadata"]["text"] for match in results["matches"]]


## Step 6–7: Same RAG Prompt & LLM call

In [ ]:
context = "\n".join(retrieved_chunks)

prompt = f"""
Use the following context to answer:

Context:
{context}

Question:
{query}

Answer:
"""

from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)